In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Attack_Prediction.csv')

In [2]:
columns=df.columns

for column in columns:
    if df[column].dtype=="int32":
        df[column]=df[column].astype("int16")
    elif df[column].dtype=="float64":
        df[column]=df[column].astype("float16")
    elif df[column].dtype=="object":
        df[column]=df[column].astype("category")

In [3]:
df['Sex']= df['Sex'].map({'Female': 0, 'Male': 1})
df['Sex']= pd.to_numeric(df['Sex'])

df['Diet']= df['Diet'].map({'Healthy': 0, 'Average': 1, 'Unhealthy':2})
df['Diet']= pd.to_numeric(df['Diet'])

df[['HBP', 'LBP']]= df['Blood Pressure'].str.split('/', expand= True)
df['HBP']= pd.to_numeric(df['HBP'])
df['LBP']= pd.to_numeric(df['LBP'])

df['Diabetes'] = df['Diabetes'].map({0: 1, 1: 0})

df['Exercise Hours Per Week']= round(df['Exercise Hours Per Week'], 0)

df['Sedentary Hours Per Day']= round(df['Sedentary Hours Per Day'], 0)

df['Income']= round(df['Income'], 0)

df['BMI']= round(df['BMI'], 0)

df = df.drop(columns=['Patient ID', 'Blood Pressure', 'Country', 'Continent', 'Hemisphere'])

In [4]:
X= df.drop(['Heart Attack Risk'], axis= 1)
y= df['Heart Attack Risk']

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 15, stratify= y)

from sklearn.preprocessing import RobustScaler
scaler= RobustScaler()
scaler.fit(X_train)

X_train= scaler.transform(X_train)
X_test= scaler.transform(X_test)

In [6]:
from imblearn.under_sampling import ClusterCentroids
from imblearn.over_sampling import SMOTE
from sklearn.cluster import KMeans

sm= ClusterCentroids(random_state= 15, estimator= KMeans(n_init= 10))
tl= SMOTE(random_state= 15)

X_sm, y_sm= sm.fit_resample(X_train, y_train)
X_sm_tl, y_sm_tl= tl.fit_resample(X_sm, y_sm)

In [7]:
from sklearn.feature_selection import SelectKBest, f_classif

selector= SelectKBest(score_func= f_classif, k= 10)
X_sm_tl_selected= selector.fit_transform(X_sm_tl, y_sm_tl)
X_sm_tl_test_selected= selector.transform(X_test)

In [8]:
feature_names= X.columns
selected_indices= selector.get_support(indices=True)
selected_features= feature_names[selected_indices]
print("Selected Features: ", selected_features)

Selected Features:  Index(['Age', 'Sex', 'Cholesterol', 'Heart Rate', 'Diabetes', 'Smoking',
       'Sedentary Hours Per Day', 'Physical Activity Days Per Week',
       'Sleep Hours Per Day', 'HBP'],
      dtype='object')


--- LogisticRegression ---

In [9]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
lr= LogisticRegression(random_state=15)
lr.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lr= lr.predict(X_sm_tl_test_selected)
y_pred_prob_lr= lr.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lr, labels= lr.classes_)
print(classification_report(y_test, y_pred_lr, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lr, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lr))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lr))

              precision    recall  f1-score   support

           0       0.65      0.47      0.55      1125
           1       0.36      0.54      0.43       628

    accuracy                           0.50      1753
   macro avg       0.51      0.51      0.49      1753
weighted avg       0.55      0.50      0.51      1753

Precision: 0.36305048335123524
Accuracy: 0.49629207073588133
AUC: 0.5043878273177635


--- DecisionTreeClassifier ---

In [10]:
from sklearn.tree import DecisionTreeClassifier
dt= DecisionTreeClassifier(random_state=15)
dt.fit(X_sm_tl_selected, y_sm_tl)
y_pred_dt= dt.predict(X_sm_tl_test_selected)
y_pred_prob_dt= dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_dt, labels= dt.classes_)
print(classification_report(y_test, y_pred_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_dt))

              precision    recall  f1-score   support

           0       0.65      0.43      0.52      1125
           1       0.36      0.58      0.45       628

    accuracy                           0.49      1753
   macro avg       0.51      0.51      0.48      1753
weighted avg       0.55      0.49      0.49      1753

Precision: 0.36427145708582837
Accuracy: 0.48659440958357103
AUC: 0.5074939844302901


--- Tuned - DecisionTreeClassifier ---

In [11]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt= GridSearchCV(estimator= dt, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_dt.best_params_)
print("Best Precision Score:", gs_dt.best_score_)

Best Parameters: {'criterion': 'entropy', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10}
Best Precision Score: 0.5928066342627496


In [12]:
tuned_dt= gs_dt.best_estimator_
y_pred_tuned_dt= tuned_dt.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_dt= tuned_dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt, labels= tuned_dt.classes_)
print(classification_report(y_test, y_pred_tuned_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt))

              precision    recall  f1-score   support

           0       0.63      0.43      0.51      1125
           1       0.35      0.54      0.43       628

    accuracy                           0.47      1753
   macro avg       0.49      0.49      0.47      1753
weighted avg       0.53      0.47      0.48      1753

Precision: 0.3486238532110092
Accuracy: 0.47233314318311465
AUC: 0.4961429582448691


--- KNeighborsClassifier ---

In [13]:
from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier()
knn.fit(X_sm_tl_selected, y_sm_tl)
y_pred_knn= knn.predict(X_sm_tl_test_selected)
y_pred_prob_knn= knn.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_knn, labels= knn.classes_)
print(classification_report(y_test, y_pred_knn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_knn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_knn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_knn))

              precision    recall  f1-score   support

           0       0.65      0.50      0.57      1125
           1       0.36      0.51      0.42       628

    accuracy                           0.50      1753
   macro avg       0.51      0.51      0.49      1753
weighted avg       0.55      0.50      0.52      1753

Precision: 0.36332574031890663
Accuracy: 0.5048488305761551
AUC: 0.5046871903750885


--- Tuned - KNeighborsClassifier ---

In [14]:
param_grid= {
             'n_neighbors': np.arange(1,40),
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan', 'minkowski']
            }

gs_knn= GridSearchCV(estimator= knn, param_grid= param_grid, cv=5, scoring= 'precision')
gs_knn.fit(X_sm_tl_selected, y_sm_tl)
print("Best Parameters:", gs_knn.best_params_)
print("Best Precision Score:", gs_knn.best_score_)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 26, 'weights': 'uniform'}
Best Precision Score: 0.5394627359921123


In [15]:
tuned_knn= gs_knn.best_estimator_
y_pred_tuned_knn= tuned_knn.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_knn= tuned_knn.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_knn, labels= tuned_knn.classes_)
print(classification_report(y_test, y_pred_tuned_knn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_knn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_knn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_knn))

              precision    recall  f1-score   support

           0       0.64      0.49      0.56      1125
           1       0.36      0.51      0.42       628

    accuracy                           0.50      1753
   macro avg       0.50      0.50      0.49      1753
weighted avg       0.54      0.50      0.51      1753

Precision: 0.3577052868391451
Accuracy: 0.49743297204791787
AUC: 0.48845293701344655


--- GaussianNB ---

In [16]:
from sklearn.naive_bayes import GaussianNB
nb= GaussianNB()
nb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_nb= nb.predict(X_sm_tl_test_selected)
y_pred_prob_nb= nb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_nb, labels= nb.classes_)
print(classification_report(y_test, y_pred_nb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_nb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_nb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_nb))

              precision    recall  f1-score   support

           0       0.66      0.34      0.45      1125
           1       0.36      0.68      0.47       628

    accuracy                           0.46      1753
   macro avg       0.51      0.51      0.46      1753
weighted avg       0.55      0.46      0.46      1753

Precision: 0.36495726495726494
Accuracy: 0.4614945807187678
AUC: 0.5110898796886058


--- SVM ---

In [17]:
from sklearn.svm import SVC
svc= SVC(kernel= 'rbf',probability= True, gamma= 1, random_state=15)
svc.fit(X_sm_tl_selected, y_sm_tl)
y_pred_svc= svc.predict(X_sm_tl_test_selected)
y_pred_prob_svc= svc.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_svc, labels= svc.classes_)
print(classification_report(y_test, y_pred_svc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_svc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_svc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_svc))

              precision    recall  f1-score   support

           0       0.65      0.48      0.55      1125
           1       0.37      0.55      0.44       628

    accuracy                           0.50      1753
   macro avg       0.51      0.51      0.50      1753
weighted avg       0.55      0.50      0.51      1753

Precision: 0.3683083511777302
Accuracy: 0.5014261266400456
AUC: 0.5162675159235669


--- Random Forest ---

In [18]:
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier(random_state=15)
rf.fit(X_sm_tl_selected, y_sm_tl)
y_pred_rf= rf.predict(X_sm_tl_test_selected)
y_pred_prob_rf= rf.predict_proba(X_sm_tl_test_selected)[:,1]

In [19]:
cm= confusion_matrix(y_test, y_pred_rf, labels= rf.classes_)
print(classification_report(y_test, y_pred_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_rf))

              precision    recall  f1-score   support

           0       0.64      0.31      0.42      1125
           1       0.36      0.69      0.47       628

    accuracy                           0.45      1753
   macro avg       0.50      0.50      0.44      1753
weighted avg       0.54      0.45      0.44      1753

Precision: 0.35831960461285006
Accuracy: 0.4455219623502567
AUC: 0.5015307855626328


In [20]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf= GridSearchCV(estimator= rf, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf.fit(X_sm_tl_selected, y_sm_tl)
print("Best Parameters:", gs_tuned_rf.best_params_)
print("Best Precision Score:", gs_tuned_rf.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 300}
Best Precision Score: 0.633830184348662


In [21]:
tuned_rf= gs_tuned_rf.best_estimator_
y_pred_tuned_rf= tuned_rf.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_rf= tuned_rf.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf, labels= tuned_rf.classes_)
print(classification_report(y_test, y_pred_tuned_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf))

              precision    recall  f1-score   support

           0       0.63      0.26      0.37      1125
           1       0.35      0.73      0.48       628

    accuracy                           0.43      1753
   macro avg       0.49      0.49      0.42      1753
weighted avg       0.53      0.43      0.41      1753

Precision: 0.35453840186190844
Accuracy: 0.4278379920136908
AUC: 0.5017537154989383


--- AdaBoost ---

In [22]:
from sklearn.ensemble import AdaBoostClassifier
ada= AdaBoostClassifier(random_state=15)
ada.fit(X_sm_tl_selected, y_sm_tl)
y_pred_ada= ada.predict(X_sm_tl_test_selected)
y_pred_prob_ada= ada.predict_proba(X_sm_tl_test_selected)[:,1]

In [23]:
cm= confusion_matrix(y_test, y_pred_ada, labels= ada.classes_)
print(classification_report(y_test, y_pred_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ada))

              precision    recall  f1-score   support

           0       0.65      0.36      0.46      1125
           1       0.36      0.66      0.47       628

    accuracy                           0.46      1753
   macro avg       0.51      0.51      0.46      1753
weighted avg       0.55      0.46      0.46      1753

Precision: 0.36347673397717295
Accuracy: 0.4643468339988591
AUC: 0.5028110403397028


In [24]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada= GridSearchCV(estimator= ada, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada.fit(X_sm_tl_selected, y_sm_tl)
print("Best Parameters:", gs_ada.best_params_)
print("Best Precision Score:", gs_ada.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 200}
Best Precision Score: 0.6448175698925586


In [25]:
tuned_ada= gs_ada.best_estimator_
y_pred_tuned_ada= tuned_ada.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_ada= tuned_ada.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada, labels= tuned_ada.classes_)
print(classification_report(y_test, y_pred_tuned_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada))

              precision    recall  f1-score   support

           0       0.68      0.22      0.34      1125
           1       0.37      0.81      0.51       628

    accuracy                           0.43      1753
   macro avg       0.52      0.52      0.42      1753
weighted avg       0.57      0.43      0.40      1753

Precision: 0.36765772298767224
Accuracy: 0.43354249857387334
AUC: 0.5033857041755131


--- GradientBoosting ---

In [26]:
from sklearn.ensemble import GradientBoostingClassifier
grb= GradientBoostingClassifier(random_state=15)
grb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_grb= grb.predict(X_sm_tl_test_selected)
y_pred_prob_grb= grb.predict_proba(X_sm_tl_test_selected)[:,1]


cm= confusion_matrix(y_test, y_pred_grb, labels= grb.classes_)
print(classification_report(y_test, y_pred_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb))

              precision    recall  f1-score   support

           0       0.64      0.15      0.24      1125
           1       0.36      0.85      0.51       628

    accuracy                           0.40      1753
   macro avg       0.50      0.50      0.37      1753
weighted avg       0.54      0.40      0.34      1753

Precision: 0.35876840696117807
Accuracy: 0.40102681118083283
AUC: 0.49514508138711966


In [27]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb= GridSearchCV(estimator= grb, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_grb.best_params_)
print("Best Precision Score:", gs_grb.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.2}
Best Precision Score: 0.6726112649941912


In [28]:
tuned_grb= gs_grb.best_estimator_
y_pred_tuned_grb= tuned_grb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_grb= tuned_grb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb, labels= tuned_grb.classes_)
print(classification_report(y_test, y_pred_tuned_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb))

              precision    recall  f1-score   support

           0       0.67      0.16      0.26      1125
           1       0.36      0.85      0.51       628

    accuracy                           0.41      1753
   macro avg       0.51      0.51      0.39      1753
weighted avg       0.56      0.41      0.35      1753

Precision: 0.36265223274695535
Accuracy: 0.41015402167712495
AUC: 0.5040368011323425


--- XGB ---

In [29]:
from xgboost import XGBClassifier
xgb= XGBClassifier(random_state=15)
xgb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_xgb= xgb.predict(X_sm_tl_test_selected)
y_pred_prob_xgb= xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb, labels= xgb.classes_)
print(classification_report(y_test, y_pred_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb))

              precision    recall  f1-score   support

           0       0.64      0.25      0.36      1125
           1       0.36      0.74      0.48       628

    accuracy                           0.43      1753
   macro avg       0.50      0.50      0.42      1753
weighted avg       0.54      0.43      0.40      1753

Precision: 0.3563218390804598
Accuracy: 0.4278379920136908
AUC: 0.505170559094126


In [30]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb= GridSearchCV(estimator= xgb, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_xgb.best_params_)
print("Best Precision Score:", gs_xgb.best_score_)

tuned_xgb= gs_xgb.best_estimator_
y_pred_tuned_xgb= tuned_xgb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_xgb= tuned_xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb, labels= tuned_xgb.classes_)
print(classification_report(y_test, y_pred_tuned_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.6682966359899265
              precision    recall  f1-score   support

           0       0.66      0.20      0.30      1125
           1       0.36      0.82      0.50       628

    accuracy                           0.42      1753
   macro avg       0.51      0.51      0.40      1753
weighted avg       0.55      0.42      0.38      1753

Precision: 0.3620933521923621
Accuracy: 0.419281232173417
AUC: 0.5010375088464261


--- LGBM ---

In [31]:
from lightgbm import LGBMClassifier
lgm= LGBMClassifier(random_state=15)
lgm.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lgm= lgm.predict(X_sm_tl_test_selected)
y_pred_prob_lgm= lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm, labels= lgm.classes_)
print(classification_report(y_test, y_pred_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm))

[LightGBM] [Info] Number of positive: 2511, number of negative: 2511
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000564 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1192
[LightGBM] [Info] Number of data points in the train set: 5022, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
              precision    recall  f1-score   support

           0       0.69      0.23      0.34      1125
           1       0.37      0.82      0.51       628

    accuracy                           0.44      1753
   macro avg       0.53      0.52      0.43      1753
weighted avg       0.58      0.44      0.40      1753

Precision: 0.37157287157287155
Accuracy: 0.43867655447803766
AUC: 0.5129087048832272


In [32]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm= GridSearchCV(estimator= lgm, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_lgm.best_params_)
print("Best Precision Score:", gs_lgm.best_score_)

tuned_lgm= gs_lgm.best_estimator_
y_pred_tuned_lgm= tuned_lgm.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_lgm= tuned_lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm, labels= tuned_lgm.classes_)
print(classification_report(y_test, y_pred_tuned_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=400, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=400
[LightGBM] [Warning] min_data_in_leaf is set=400, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=400
[LightGBM] [Info] Number of positive: 2511, number of negative: 2511
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000345 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1192
[LightGBM] [Info] Number of data points in the train set: 5022, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits w

--- CatBoost ---

In [33]:
from catboost import CatBoostClassifier
cat= CatBoostClassifier(random_state=15)
cat.fit(X_sm_tl_selected, y_sm_tl)
y_pred_cat= cat.predict(X_sm_tl_test_selected)
y_pred_prob_cat= cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_cat, labels= cat.classes_)
print(classification_report(y_test, y_pred_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat))

Learning rate set to 0.020521
0:	learn: 0.6887950	total: 148ms	remaining: 2m 27s
1:	learn: 0.6851298	total: 157ms	remaining: 1m 18s
2:	learn: 0.6811345	total: 167ms	remaining: 55.5s
3:	learn: 0.6774913	total: 177ms	remaining: 44s
4:	learn: 0.6747476	total: 184ms	remaining: 36.7s
5:	learn: 0.6707685	total: 192ms	remaining: 31.8s
6:	learn: 0.6675755	total: 199ms	remaining: 28.2s
7:	learn: 0.6655603	total: 207ms	remaining: 25.6s
8:	learn: 0.6630988	total: 213ms	remaining: 23.5s
9:	learn: 0.6606940	total: 221ms	remaining: 21.9s
10:	learn: 0.6579462	total: 227ms	remaining: 20.4s
11:	learn: 0.6557935	total: 233ms	remaining: 19.2s
12:	learn: 0.6542911	total: 239ms	remaining: 18.2s
13:	learn: 0.6518956	total: 244ms	remaining: 17.2s
14:	learn: 0.6505430	total: 250ms	remaining: 16.4s
15:	learn: 0.6489175	total: 257ms	remaining: 15.8s
16:	learn: 0.6474532	total: 262ms	remaining: 15.2s
17:	learn: 0.6454742	total: 268ms	remaining: 14.6s
18:	learn: 0.6439325	total: 273ms	remaining: 14.1s
19:	learn: 

200:	learn: 0.5467724	total: 895ms	remaining: 3.56s
201:	learn: 0.5465055	total: 899ms	remaining: 3.55s
202:	learn: 0.5461260	total: 902ms	remaining: 3.54s
203:	learn: 0.5459687	total: 905ms	remaining: 3.53s
204:	learn: 0.5454504	total: 909ms	remaining: 3.52s
205:	learn: 0.5451729	total: 912ms	remaining: 3.51s
206:	learn: 0.5449257	total: 915ms	remaining: 3.51s
207:	learn: 0.5447057	total: 918ms	remaining: 3.5s
208:	learn: 0.5440922	total: 922ms	remaining: 3.49s
209:	learn: 0.5438218	total: 925ms	remaining: 3.48s
210:	learn: 0.5436323	total: 928ms	remaining: 3.47s
211:	learn: 0.5433147	total: 931ms	remaining: 3.46s
212:	learn: 0.5428361	total: 934ms	remaining: 3.45s
213:	learn: 0.5424718	total: 937ms	remaining: 3.44s
214:	learn: 0.5422260	total: 940ms	remaining: 3.43s
215:	learn: 0.5420652	total: 944ms	remaining: 3.42s
216:	learn: 0.5417029	total: 947ms	remaining: 3.42s
217:	learn: 0.5415062	total: 951ms	remaining: 3.41s
218:	learn: 0.5412591	total: 954ms	remaining: 3.4s
219:	learn: 0.

385:	learn: 0.5018197	total: 1.6s	remaining: 2.55s
386:	learn: 0.5016816	total: 1.61s	remaining: 2.55s
387:	learn: 0.5014975	total: 1.62s	remaining: 2.55s
388:	learn: 0.5012901	total: 1.62s	remaining: 2.54s
389:	learn: 0.5011432	total: 1.63s	remaining: 2.54s
390:	learn: 0.5009763	total: 1.63s	remaining: 2.54s
391:	learn: 0.5007995	total: 1.64s	remaining: 2.54s
392:	learn: 0.5004580	total: 1.64s	remaining: 2.53s
393:	learn: 0.5002453	total: 1.64s	remaining: 2.53s
394:	learn: 0.5000267	total: 1.65s	remaining: 2.52s
395:	learn: 0.4997592	total: 1.65s	remaining: 2.52s
396:	learn: 0.4995385	total: 1.66s	remaining: 2.52s
397:	learn: 0.4993589	total: 1.66s	remaining: 2.51s
398:	learn: 0.4991670	total: 1.67s	remaining: 2.51s
399:	learn: 0.4989823	total: 1.67s	remaining: 2.51s
400:	learn: 0.4987198	total: 1.67s	remaining: 2.5s
401:	learn: 0.4985326	total: 1.68s	remaining: 2.5s
402:	learn: 0.4983355	total: 1.68s	remaining: 2.49s
403:	learn: 0.4979996	total: 1.69s	remaining: 2.49s
404:	learn: 0.4

571:	learn: 0.4609277	total: 2.49s	remaining: 1.87s
572:	learn: 0.4608021	total: 2.5s	remaining: 1.86s
573:	learn: 0.4605943	total: 2.5s	remaining: 1.86s
574:	learn: 0.4603488	total: 2.51s	remaining: 1.85s
575:	learn: 0.4601570	total: 2.51s	remaining: 1.85s
576:	learn: 0.4599948	total: 2.52s	remaining: 1.85s
577:	learn: 0.4597716	total: 2.52s	remaining: 1.84s
578:	learn: 0.4596175	total: 2.53s	remaining: 1.84s
579:	learn: 0.4594778	total: 2.53s	remaining: 1.83s
580:	learn: 0.4592664	total: 2.54s	remaining: 1.83s
581:	learn: 0.4591049	total: 2.54s	remaining: 1.82s
582:	learn: 0.4589077	total: 2.55s	remaining: 1.82s
583:	learn: 0.4586662	total: 2.55s	remaining: 1.82s
584:	learn: 0.4584575	total: 2.56s	remaining: 1.81s
585:	learn: 0.4582400	total: 2.56s	remaining: 1.81s
586:	learn: 0.4580388	total: 2.57s	remaining: 1.81s
587:	learn: 0.4578147	total: 2.58s	remaining: 1.81s
588:	learn: 0.4575998	total: 2.58s	remaining: 1.8s
589:	learn: 0.4573861	total: 2.59s	remaining: 1.8s
590:	learn: 0.45

756:	learn: 0.4258666	total: 3.38s	remaining: 1.09s
757:	learn: 0.4257574	total: 3.39s	remaining: 1.08s
758:	learn: 0.4255626	total: 3.39s	remaining: 1.08s
759:	learn: 0.4254154	total: 3.4s	remaining: 1.07s
760:	learn: 0.4252332	total: 3.4s	remaining: 1.07s
761:	learn: 0.4250645	total: 3.4s	remaining: 1.06s
762:	learn: 0.4248859	total: 3.41s	remaining: 1.06s
763:	learn: 0.4246871	total: 3.41s	remaining: 1.05s
764:	learn: 0.4246031	total: 3.42s	remaining: 1.05s
765:	learn: 0.4244294	total: 3.42s	remaining: 1.04s
766:	learn: 0.4241924	total: 3.42s	remaining: 1.04s
767:	learn: 0.4240631	total: 3.43s	remaining: 1.03s
768:	learn: 0.4238592	total: 3.43s	remaining: 1.03s
769:	learn: 0.4236766	total: 3.43s	remaining: 1.02s
770:	learn: 0.4235173	total: 3.44s	remaining: 1.02s
771:	learn: 0.4234008	total: 3.44s	remaining: 1.02s
772:	learn: 0.4231478	total: 3.45s	remaining: 1.01s
773:	learn: 0.4230126	total: 3.45s	remaining: 1.01s
774:	learn: 0.4226879	total: 3.45s	remaining: 1s
775:	learn: 0.4225

956:	learn: 0.3948398	total: 4.09s	remaining: 184ms
957:	learn: 0.3947594	total: 4.1s	remaining: 180ms
958:	learn: 0.3946003	total: 4.1s	remaining: 175ms
959:	learn: 0.3944590	total: 4.1s	remaining: 171ms
960:	learn: 0.3943189	total: 4.11s	remaining: 167ms
961:	learn: 0.3941601	total: 4.11s	remaining: 162ms
962:	learn: 0.3940451	total: 4.11s	remaining: 158ms
963:	learn: 0.3938879	total: 4.12s	remaining: 154ms
964:	learn: 0.3937582	total: 4.12s	remaining: 149ms
965:	learn: 0.3936052	total: 4.12s	remaining: 145ms
966:	learn: 0.3934787	total: 4.13s	remaining: 141ms
967:	learn: 0.3933129	total: 4.13s	remaining: 136ms
968:	learn: 0.3931448	total: 4.13s	remaining: 132ms
969:	learn: 0.3930056	total: 4.13s	remaining: 128ms
970:	learn: 0.3928634	total: 4.14s	remaining: 124ms
971:	learn: 0.3926638	total: 4.14s	remaining: 119ms
972:	learn: 0.3925483	total: 4.14s	remaining: 115ms
973:	learn: 0.3924207	total: 4.15s	remaining: 111ms
974:	learn: 0.3923090	total: 4.15s	remaining: 106ms
975:	learn: 0.3

In [34]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat= GridSearchCV(estimator= cat, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_cat.best_params_)
print("Best Precision Score:", gs_cat.best_score_)

tuned_cat= gs_cat.best_estimator_
y_pred_tuned_cat= tuned_cat.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_cat= tuned_cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat, labels= tuned_cat.classes_)
print(classification_report(y_test, y_pred_tuned_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.6518786	total: 2.16ms	remaining: 214ms
1:	learn: 0.6454470	total: 4.16ms	remaining: 204ms
2:	learn: 0.6365103	total: 6.11ms	remaining: 198ms
3:	learn: 0.6234536	total: 8.37ms	remaining: 201ms
4:	learn: 0.6131812	total: 10.5ms	remaining: 199ms
5:	learn: 0.6074506	total: 12.5ms	remaining: 195ms
6:	learn: 0.5946728	total: 14.4ms	remaining: 192ms
7:	learn: 0.5867062	total: 16.6ms	remaining: 191ms
8:	learn: 0.5843421	total: 18.3ms	remaining: 185ms
9:	learn: 0.5809571	total: 20ms	remaining: 180ms
10:	learn: 0.5747627	total: 21.6ms	remaining: 175ms
11:	learn: 0.5683339	total: 23.5ms	remaining: 172ms
12:	learn: 0.5649874	total: 24.8ms	remaining: 166ms
13:	learn: 0.5546504	total: 26.4ms	remaining: 162ms
14:	learn: 0.5530079	total: 27.8ms	remaining: 157ms
15:	learn: 0.5514407	total: 29.3ms	remaining: 154ms
16:	learn: 0.5462700	total: 30.9ms	remaining: 151ms
17:	learn: 0.5449822	total: 32.6ms	remaining: 149ms
18:	learn: 0.5

--- Visualisation ---

In [35]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt, zero_division= 0),
                    'KNeighborsClassifier Precision:': precision_score(y_test, y_pred_knn, zero_division= 0),
                    'Tuned KNeighborsClassifier Precision:': precision_score(y_test, y_pred_tuned_knn, zero_division= 0),
                    'GaussianNB Precision:': precision_score(y_test, y_pred_nb, zero_division= 0),
                    'SVM Precision:': precision_score(y_test, y_pred_svc, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat, zero_division= 0)
                  }

annova_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
annova_precision= annova_precision.sort_values(by= 'Precision Score', ascending=False)
print(annova_precision)

                                    Model  Precision Score
15                        LGBM Precision:         0.371573
16                  Tuned LGBM Precision:         0.368885
6                          SVM Precision:         0.368308
10              Tuned AdaBoost Precision:         0.367658
5                   GaussianNB Precision:         0.364957
1                Decision Tree Precision:         0.364271
9                     AdaBoost Precision:         0.363477
3         KNeighborsClassifier Precision:         0.363326
0          Logistic Regression Precision:         0.363050
12      Tuned GradientBoosting Precision:         0.362652
17                    CatBoost Precision:         0.362585
14                   Tuned XGB Precision:         0.362093
11            GradientBoosting Precision:         0.358768
7                Random Forest Precision:         0.358320
4   Tuned KNeighborsClassifier Precision:         0.357705
13                         XGB Precision:         0.3563

In [36]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt),
                    'KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_knn),
                    'Tuned KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_tuned_knn),
                    'GaussianNB Accuracy:': accuracy_score(y_test, y_pred_nb),
                    'SVM Accuracy:': accuracy_score(y_test, y_pred_svc),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat)
                  }

annova_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
annova_accuracy= annova_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(annova_accuracy)

                                   Model  Accuracy Score
3         KNeighborsClassifier Accuracy:        0.504849
6                          SVM Accuracy:        0.501426
4   Tuned KNeighborsClassifier Accuracy:        0.497433
0          Logistic Regression Accuracy:        0.496292
1                Decision Tree Accuracy:        0.486594
2          Tuned Decision Tree Accuracy:        0.472333
9                     AdaBoost Accuracy:        0.464347
5                   GaussianNB Accuracy:        0.461495
16                  Tuned LGBM Accuracy:        0.455220
7                Random Forest Accuracy:        0.445522
15                        LGBM Accuracy:        0.438677
10              Tuned AdaBoost Accuracy:        0.433542
8          Tuned Random Forest Accuracy:        0.427838
13                         XGB Accuracy:        0.427838
14                   Tuned XGB Accuracy:        0.419281
18              Tuned CatBoost Accuracy:        0.419281
17                    CatBoost 

In [37]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt),
                    'KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_knn),
                    'Tuned KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_tuned_knn),
                    'GaussianNB AUC:': roc_auc_score(y_test, y_pred_prob_nb),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat)
                  }

annova_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
annova_auc= annova_auc.sort_values(by= 'AUC Score', ascending=False)
print(annova_auc)

                              Model  AUC Score
16                  Tuned LGBM AUC:   0.525529
6                          SVM AUC:   0.516268
15                        LGBM AUC:   0.512909
5                   GaussianNB AUC:   0.511090
17                    CatBoost AUC:   0.507832
1                Decision Tree AUC:   0.507494
13                         XGB AUC:   0.505171
3         KNeighborsClassifier AUC:   0.504687
0          Logistic Regression AUC:   0.504388
12      Tuned GradientBoosting AUC:   0.504037
10              Tuned AdaBoost AUC:   0.503386
9                     AdaBoost AUC:   0.502811
18              Tuned CatBoost AUC:   0.502282
8          Tuned Random Forest AUC:   0.501754
7                Random Forest AUC:   0.501531
14                   Tuned XGB AUC:   0.501038
2          Tuned Decision Tree AUC:   0.496143
11            GradientBoosting AUC:   0.495145
4   Tuned KNeighborsClassifier AUC:   0.488453
